In [1]:
!pip install transformers datasets torch pandas evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [26]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from datasets import load_dataset

squad = load_dataset("rajpurkar/squad_v2", split="train[:5000]")

README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [4]:
squad

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 5000
})

In [5]:
squad = squad.train_test_split(test_size=0.2)

In [6]:
squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1000
    })
})

In [36]:
squad['test'][0]

{'id': '56d5357b2593cc1400307ad9',
 'title': '2008_Sichuan_earthquake',
 'context': 'Details of school casualties had been under non-governmental investigation since December 2008 by volunteers including artist and architect Ai Weiwei, who had been constantly posting updates on his blog since March 2009. The official tally of students killed in the earthquake was not released until May 7, 2009, almost a year after the earthquake. According to the state-run Xinhua news agency, the earthquake killed 5,335 students and left another 546 children disabled. In the aftermath of the earthquake, the Chinese government declared that parents who had lost their only children would get free treatment from fertility clinics to reverse vasectomies and tubal ligations conducted by family planning authorities.',
 'question': 'What date were the official numbers of students killed in the earthquake released?',
 'answers': {'text': ['May 7, 2009'], 'answer_start': [300]}}

In [7]:
squad["train"][0]

{'id': '56cf50b2aab44d1400b88fbd',
 'title': 'Frédéric_Chopin',
 'context': 'Chopin has figured extensively in Polish literature, both in serious critical studies of his life and music and in fictional treatments. The earliest manifestation was probably an 1830 sonnet on Chopin by Leon Ulrich. French writers on Chopin (apart from Sand) have included Marcel Proust and André Gide; and he has also featured in works of Gottfried Benn and Boris Pasternak. There are numerous biographies of Chopin in English (see bibliography for some of these).',
 'question': 'An 1830 sonnet was written about Chopin by what man?',
 'answers': {'text': ['Leon Ulrich'], 'answer_start': [205]}}

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Wikidepia/indobert-lite-squad")

tokenizer_config.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/225k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [14]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]

        # Check if 'answer_start' and 'text' are not empty
        if not answer["answer_start"] or not answer["text"]:
            # No valid answer, assign (0, 0) as start and end positions
            start_positions.append(0)
            end_positions.append(0)
            continue

        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while idx < len(sequence_ids) and sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


In [15]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [16]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [17]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("Wikidepia/indobert-lite-squad")

pytorch_model.bin:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

In [30]:
training_args = TrainingArguments(
    output_dir="QA-lite-squad",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,2.060822
2,0.384500,2.238172
3,0.384500,2.437726


TrainOutput(global_step=750, training_loss=0.29865129343668617, metrics={'train_runtime': 885.7007, 'train_samples_per_second': 13.549, 'train_steps_per_second': 0.847, 'total_flos': 198754228224000.0, 'train_loss': 0.29865129343668617, 'epoch': 3.0})

In [31]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/osmanh/QA-lite-squad/commit/29fe515b7b9f3e9fd21af8be6b0db0e5c1117409', commit_message='End of training', commit_description='', oid='29fe515b7b9f3e9fd21af8be6b0db0e5c1117409', pr_url=None, repo_url=RepoUrl('https://huggingface.co/osmanh/QA-lite-squad', endpoint='https://huggingface.co', repo_type='model', repo_id='osmanh/QA-lite-squad'), pr_revision=None, pr_num=None)

In [37]:
question = "What date were the official numbers of students killed in the earthquake released?"
context = "Details of school casualties had been under non-governmental investigation since December 2008 by volunteers including artist and architect Ai Weiwei, who had been constantly posting updates on his blog since March 2009. The official tally of students killed in the earthquake was not released until May 7, 2009, almost a year after the earthquake. According to the state-run Xinhua news agency, the earthquake killed 5,335 students and left another 546 children disabled. In the aftermath of the earthquake, the Chinese government declared that parents who had lost their only children would get free treatment from fertility clinics to reverse vasectomies and tubal ligations conducted by family planning authorities."

In [38]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="osmanh/QA-lite-squad")
question_answerer(question=question, context=context)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'score': 0.9921325445175171,
 'start': 300,
 'end': 311,
 'answer': 'May 7, 2009'}